In [135]:
import pandas as pd
import numpy as np
from fuzzywuzzy import process

In [136]:
religious_data = pd.read_excel("2020_USRC_Group_Detail.xlsx")

religious_data = religious_data.loc[(religious_data["State Name"] == "Texas") | (religious_data["State Name"] == "Florida")]

In [137]:
# religious_data = religious_data.drop(columns=["Congregations", "Adherents"])
religious_data.loc[(religious_data["County Name"] == "Duval County") & (religious_data["State Name"] == "Florida")]["Adherents as % of Total Adherents"].sum()

1.0

In [138]:
def mode(series):
    return series.mode().iloc[0]

florida_final_data = religious_data.loc[religious_data["State Name"] == "Florida"]
texas_final_data = religious_data.loc[religious_data["State Name"] == "Texas"]

total_adherents_florida = florida_final_data.groupby('County Name').agg({
    'FIPS': mode,
    'State Name': mode,
    'Group Code': lambda x: 0,
    'Group Name': lambda x: 'Non-Religious',
    'Adherents as % of Total Adherents': lambda x: 0,
    'Adherents as % of Total Population': 'sum'
}).reset_index()

total_adherents_texas = texas_final_data.groupby('County Name').agg({
    'FIPS': mode,
    'State Name': mode,
    'Group Code': lambda x: 0,
    'Group Name': lambda x: 'Non-Religious',
    'Adherents as % of Total Adherents': lambda x: 0,
    'Adherents as % of Total Population': 'sum'
}).reset_index()

# Calculate the percentage of non-religious or atheist population
total_adherents_texas["Adherents as % of Total Population"] = 1 - total_adherents_texas["Adherents as % of Total Population"]
total_adherents_florida["Adherents as % of Total Population"] = 1 - total_adherents_florida["Adherents as % of Total Population"]

# Append the new rows to the original dataframe
df_texas = pd.concat([religious_data, total_adherents_texas], ignore_index=True)
df_florida = pd.concat([religious_data, total_adherents_florida], ignore_index=True)

In [139]:
# religious_data = df.copy()
# religious_grouped = religious_data.groupby(by="Group Name")["Adherents as % of Total Population"].sum().reset_index()
# religious_grouped

In [140]:
# religious_grouped = religious_grouped.sort_values(by="Adherents as % of Total Population", ascending=False)
# religious_grouped.head(10)
# # religious_data["Group Name"].unique()

In [141]:
# Top 5 most common
common_religions = ["Non-Religious", "Southern Baptist Convention", "Catholic Church", "Non-denominational Christian Churches", "United Methodist Church"]

final_data_tx = df_texas.loc[df_texas["Group Name"].isin(common_religions)]
final_data_fa = df_florida.loc[df_florida["Group Name"].isin(common_religions)]

pivot_table_texas = pd.pivot_table(final_data_tx, values='Adherents as % of Total Population', index='County Name', columns='Group Name', aggfunc='sum', fill_value=0)
pivot_table_florida = pd.pivot_table(final_data_fa, values='Adherents as % of Total Population', index='County Name', columns='Group Name', aggfunc='sum', fill_value=0)

# pivot_table_florida.to_csv("Religious_Breakdown_Florida.csv")
# # pivot_table_texas.to_csv("Religious_Breakdown_Texas.csv")
pivot_table_florida = pivot_table_florida.reset_index()
pivot_table_texas = pivot_table_texas.reset_index()

In [142]:
florida_data = pd.read_csv("final_florida_df.csv")
florida_data

,Unnamed: 0,LEA_STATE_NAME,LEAID,LEA_NAME,banned_books,LEA_ENR,SCH_FTETEACH_TOT,SATACT_P,APENR_P,P_EXPULSIONS,...,TOT_SEX_HB,NMCNTY,Median Household Income,PVI_parsed,County Name,Catholic Church,Non-Religious,Non-denominational Christian Churches,Southern Baptist Convention,United Methodist Church
0,0,FLORIDA,1200002,FL VIRTUAL,False,6748,0.008595,7.898637,5.853586,0.000000,...,NaN,Orange County,55071,-0.766667,Orange County,0.283870,0.508643,0.171242,0.319039,0.037226
1,1,FLORIDA,1200030,ALACHUA,False,29737,0.057114,7.694118,8.037125,0.016814,...,NaN,Alachua County,57729,-0.766667,Alachua County,0.098378,0.535218,0.122635,0.073323,0.046160
2,2,FLORIDA,1200060,BAKER,False,5044,0.058541,8.068993,0.733545,0.693894,...,NaN,Baker County,52358,0.800000,Baker County,0.022612,0.667716,0.131286,0.095828,0.011536
3,3,FLORIDA,1200090,BAY,False,28073,0.061955,7.020981,7.929327,0.295658,...,NaN,Bay County,54430,0.400000,Bay County,0.065782,0.631421,0.065405,0.124292,0.026858
4,4,FLORIDA,1200120,BRADFORD,False,3214,0.053843,6.969508,2.955818,0.311139,...,NaN,Bradford County,44411,0.533333,Bradford County,0.005158,0.593435,0.027206,0.215560,0.053210
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,68,FLORIDA,1202012,FAU LAB SCH,False,2555,0.058603,7.632094,NaN,0.000000,...,NaN,Palm Beach County,63493,-0.633333,Palm Beach County,0.191649,0.605608,0.053914,0.024650,0.007866
69,69,FLORIDA,1202013,FSU LAB SCH,False,2405,0.057838,7.983368,11.018711,0.083160,...,NaN,Leon County,68504,-0.833333,Leon County,0.123268,0.511246,0.209763,0.289471,0.088808
70,70,FLORIDA,1202014,FAMU LAB SCH,False,586,0.073379,8.020478,1.877133,0.000000,...,NaN,Leon County,68504,-0.833333,Leon County,0.123268,0.511246,0.209763,0.289471,0.088808
71,71,FLORIDA,1202015,UF LAB SCH,False,1153,0.052905,14.397225,22.549870,0.000000,...,NaN,Alachua County,57729,-0.766667,Alachua County,0.098378,0.535218,0.122635,0.073323,0.046160


In [143]:
merged_florida = pd.merge(florida_data, pivot_table_florida, left_on='NMCNTY', right_on='County Name', how="left")
merged_florida = merged_florida.drop(columns="Unnamed: 0")
merged_florida.to_csv("Final_florida_df.csv")

In [144]:
texas_final_data

,FIPS,State Name,County Name,Group Code,Group Name,Congregations,Adherents,Adherents as % of Total Adherents,Adherents as % of Total Population
66596,48001,Texas,Anderson County,003,African Methodist Episcopal Church,4.0,706.0,0.022738,0.012189
66597,48001,Texas,Anderson County,017,American Baptist Association,1.0,206.0,0.006635,0.003557
66598,48001,Texas,Anderson County,053,Assemblies of God,7.0,1051.0,0.033850,0.018145
66599,48001,Texas,Anderson County,056,Baha'i Faith USA,0.0,6.0,0.000193,0.000104
66600,48001,Texas,Anderson County,059,Baptist Missionary Association of America,4.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
71867,48507,Texas,Zavala County,133,Church of God (Seventh Day),1.0,NaN,NaN,NaN
71868,48507,Texas,Zavala County,167,Churches of Christ,1.0,19.0,0.002036,0.001965
71869,48507,Texas,Zavala County,500,Non-denominational Christian Churches,1.0,78.0,0.008360,0.008066
71870,48507,Texas,Zavala County,419,Southern Baptist Convention,4.0,354.0,0.037942,0.036608


In [145]:
texas_table = pd.read_csv("Final_DF.csv")
texas_table = texas_table.drop(columns=["Unnamed: 0"])
texas_table

,LEA_STATE_NAME,LEAID,LEA_NAME,LEA_ENR,banned_books,SCH_FTETEACH_TOT,NMCNTY,County Name,overall_TPI,Closest_Match,Median Household Income,School District,dist_key
0,TEXAS,4800001,CROSBYTON CISD,365,False,0.101370,crosby,crosby,0.740,CROSBY,60382.0,"Crosby Independent School District / 27,185",CROSBY
1,TEXAS,4800002,SPUR ISD,261,False,0.107280,dickens,dickens,0.865,SPUR,37356.0,"Spur Independent School District / 1,935",SPUR
2,TEXAS,4800003,ROCKSPRINGS ISD,313,False,0.076677,edwards,edwards,0.850,ROCKSPRINGS,41429.0,"Rocksprings Independent School District / 1,711",ROCKSPRINGS
3,TEXAS,4800004,POR VIDA ACADEMY,273,False,0.091575,bexar,bexar,-0.570,ACADEMY,62772.0,"Academy Independent School District / 5,003",ACADEMY
4,TEXAS,4800005,BLACKWELL CISD,155,False,0.129032,mitchell,mitchell,0.855,BLACKWELL CONSOLIDATED,46944.0,Blackwell Consolidated Independent School Dist...,BLACKWELL CONSOLIDATED
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1211,TEXAS,4846680,YSLETA ISD,41198,False,0.066469,el paso,el paso,-0.685,YSLETA,36079.0,"Ysleta Independent School District / 196,164",YSLETA
1212,TEXAS,4846710,ZAPATA COUNTY ISD,3521,False,0.062482,zapata,zapata,-0.585,ZAPATA COUNTY,30758.0,Zapata County Independent School District / 14...,ZAPATA COUNTY
1213,TEXAS,4846740,ZAVALLA ISD,368,False,0.092391,angelina,angelina,0.750,ZAVALLA,38684.0,"Zavalla Independent School District / 2,795",ZAVALLA
1214,TEXAS,4846770,ZEPHYR ISD,202,False,0.103960,brown,brown,0.875,ZEPHYR,51250.0,Zephyr Independent School District / 817,ZEPHYR


In [147]:
pivot_table_texas['County Name'] = pivot_table_texas['County Name'].str.replace('County', '').str.strip().str.lower()
merged_texas = pd.merge(texas_table, pivot_table_texas, left_on='NMCNTY', right_on='County Name', how="left")
merged_texas = merged_florida.drop(columns=["County Name_x", "County Name_y", ""])
merged_texas

,LEA_STATE_NAME,LEAID,LEA_NAME,LEA_ENR,banned_books,SCH_FTETEACH_TOT,NMCNTY,County Name_x,overall_TPI,Closest_Match,Median Household Income,School District,dist_key,County Name_y,Catholic Church,Non-Religious,Non-denominational Christian Churches,Southern Baptist Convention,United Methodist Church
0,TEXAS,4800001,CROSBYTON CISD,365,False,0.101370,crosby,crosby,0.740,CROSBY,60382.0,"Crosby Independent School District / 27,185",CROSBY,crosby,0.319696,0.221118,0.009741,0.302942,0.073252
1,TEXAS,4800002,SPUR ISD,261,False,0.107280,dickens,dickens,0.865,SPUR,37356.0,"Spur Independent School District / 1,935",SPUR,dickens,0.167797,0.229379,0.062147,0.287006,0.115254
2,TEXAS,4800003,ROCKSPRINGS ISD,313,False,0.076677,edwards,edwards,0.850,ROCKSPRINGS,41429.0,"Rocksprings Independent School District / 1,711",ROCKSPRINGS,edwards,0.347398,-0.035865,0.281294,0.177215,0.134318
3,TEXAS,4800004,POR VIDA ACADEMY,273,False,0.091575,bexar,bexar,-0.570,ACADEMY,62772.0,"Academy Independent School District / 5,003",ACADEMY,bexar,0.275547,0.469977,0.095839,0.046129,0.016119
4,TEXAS,4800005,BLACKWELL CISD,155,False,0.129032,mitchell,mitchell,0.855,BLACKWELL CONSOLIDATED,46944.0,Blackwell Consolidated Independent School Dist...,BLACKWELL CONSOLIDATED,mitchell,0.349722,0.257175,0.000000,0.283204,0.051613
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1211,TEXAS,4846680,YSLETA ISD,41198,False,0.066469,el paso,el paso,-0.685,YSLETA,36079.0,"Ysleta Independent School District / 196,164",YSLETA,el paso,0.479678,0.376156,0.057995,0.021532,0.003295
1212,TEXAS,4846710,ZAPATA COUNTY ISD,3521,False,0.062482,zapata,zapata,-0.585,ZAPATA COUNTY,30758.0,Zapata County Independent School District / 14...,ZAPATA COUNTY,zapata,0.549788,0.349629,0.015840,0.031032,0.004104
1213,TEXAS,4846740,ZAVALLA ISD,368,False,0.092391,angelina,angelina,0.750,ZAVALLA,38684.0,"Zavalla Independent School District / 2,795",ZAVALLA,angelina,0.139939,0.296533,0.070606,0.210660,0.040870
1214,TEXAS,4846770,ZEPHYR ISD,202,False,0.103960,brown,brown,0.875,ZEPHYR,51250.0,Zephyr Independent School District / 817,ZEPHYR,brown,0.039165,0.375850,0.083581,0.349127,0.077779
